# ACL debugging with Batfish

Network engineers are responsible for verifying that the ACLs in their networks are permitting and denying traffic as intended. This generally requires loading each ACL onto a lab device in order to test its behavior on a wide range of packets. Batfish makes it easy to see what each ACL will do with any packet, right down to the line of the ACL that matches it. We also provide a sanity check to ensure that every line in every ACL can match some packet that won't be matched by a previous line.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/acl-debugging?pixel&useReferer)In this notebook, we will start by verifying that `host1` in our example network is correctly configured as a DNS server, then go on to check that all ACL lines are reachable across the network. The example network is shown below.

![example-network](https://raw.githubusercontent.com/batfish/pybatfish/master/jupyter_notebooks/networks/example/example-network.png)

`SNAPSHOT_PATH` below can be updated to point to a custom snapshot directory, see the [Batfish instructions](https://github.com/batfish/batfish/wiki/Packaging-snapshots-for-analysis) for how to package data for analysis.<br>
More example networks are available in the [networks](https://github.com/batfish/batfish/tree/master/networks) folder of the Batfish repository.

In [1]:
# Import packages and load questions
%run startup.py

# Initialize a network and snapshot
NETWORK_NAME = "acl_debugging_network"
SNAPSHOT_NAME = "example_snapshot"

SNAPSHOT_PATH = "networks/example"

bf_set_network(NETWORK_NAME)
bf_init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

/Users/corinaminer/Batfish/pybatfish/pybatfish/client/commands.py:48: UserWarning: Pybatfish public API is being updated, note that API names and parameters will soon change.
  "Pybatfish public API is being updated, note that API names and parameters will soon change.")


'example_snapshot'

## Debugging how ACLs treat a given packet

In this notebook we're operating from the perspective of `AS1` and would like to confirm that our ACLs are correctly configured to enable a DNS service hosted on `host1`. Packets from `AS1` destined for `host1` will hit two ACLs:
- Entering `AS2`, they will hit an ACL on the border router called `OUTSIDE_TO_INSIDE`
- Entering `host1`, they will hit an input ACL `filter::INPUT`

The Test Filters question (`bfq.testfilters`) is helpful for checking what ACLs do with particular packets. We can use it to test that ACLs `OUTSIDE_TO_INSIDE` and `filter::INPUT` permit DNS packets destined for `host1`.

### Parameters of Test Filters

The Test Filters question takes in the properties of a flow and a set of ACLs to test. The answer shows what each ACL will do to that flow and why.

To specify the flow, you must provide the source IP address `srcIp` and the destination `dst` (hostname or IP address). Optionally, you can also specify IP protocols, ports, TCP flags, ICMP codes, and other properties ([see documentation for details](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.testfilters)). The question will fill in any unspecified flow properties with default values.

The set of ACLs to examine can be narrowed down using these optional parameters:
- `nodes` specifies the ACLs present on the set of nodes matching the given regex
- `filters` specifies ACLs with names matching the given regex

If `nodes` and `filters` are left blank, Test Filters will give results for every ACL in the network.

#### Tip: Pull up a table of IP addresses to help fill in `srcIp` and `dst`
The IP Owners question achieves this nicely (we'll use [Pandas APIs](https://pandas.pydata.org/pandas-docs/stable/) here and throughout the notebook to filter results):

In [2]:
# Show IP Owners for host1 and devices in AS1
ip_owners = bfq.ipOwners().answer().frame()
ip_owners[ip_owners['Node'].apply(lambda hostname: 'as1' in hostname or hostname == 'host1')]

,Node,VRF,Interface,IP,Mask,Active
8,as1border1,default,GigabitEthernet1/0,10.12.11.1,24,True
14,as1border2,default,GigabitEthernet0/0,10.13.22.1,24,True
15,as1border1,default,GigabitEthernet0/0,1.0.1.1,24,True
20,as1border1,default,Loopback0,1.1.1.1,32,True
22,as1border2,default,GigabitEthernet1/0,1.0.2.1,24,True
27,as1border2,default,Loopback0,1.2.2.2,32,True
28,as1core1,default,Loopback0,1.10.1.1,32,True
33,as1border2,default,GigabitEthernet2/0,10.14.22.1,24,True
45,as1core1,default,GigabitEthernet0/0,1.0.2.2,24,True
46,host1,default,eth0,2.128.0.101,24,True


### Verifying that ACLs will not block DNS requests to `host1`

Suppose `as1core1` sends a DNS request destined for DNS server `host1`. Let's first run Test Filters on ACL `OUTSIDE_TO_INSIDE` to check that it will permit such a packet.

The result shows that the second line of `OUTSIDE_TO_INSIDE` matches and permits DNS packets. Since that line, `permit ip any any`, will also permit many packets that should not reach `host1`, let's also check ACL `filter::INPUT` on `host1`.

In [3]:
as1core1_ip = "1.0.1.2"
host1_ip = "2.128.0.101"
node = "as2border1"
acl = "OUTSIDE_TO_INSIDE"
bfq.testfilters(headers=HeaderConstraints(dstIps=host1_ip, srcIps=as1core1_ip, applications=["dns"]), nodes=node, filters=acl).answer().frame()

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,as2border1,OUTSIDE_TO_INSIDE,start=as2border1 [1.0.1.2:49152->2.128.0.101:53 UDP],PERMIT,permit ip any any,"Flow permitted by 'extended ipv4 access-list' named 'OUTSIDE_TO_INSIDE', index 2: permit ip any any"


Below is a similar Test Filters result on the input ACL to `host1`. As shown, that filter also permits DNS packets, and the matching line is much more narrowly targeted: it permits UDP traffic to port 53 (i.e. DNS traffic).

In [4]:
node = "host1"
acl = "filter::INPUT"
bfq.testfilters(headers=HeaderConstraints(dstIps=host1_ip, srcIps=as1core1_ip, applications=["dns"]), nodes=node, filters=acl).answer().frame()

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,host1,filter::INPUT,start=host1 [1.0.1.2:49152->2.128.0.101:53 UDP],PERMIT,-p udp --dport 53 -j ACCEPT,"Flow permitted by ACL named 'filter::INPUT', index 0: -p udp --dport 53 -j ACCEPT"


### Checking that an HTTP packet will be dropped
Meanwhile, other traffic from `AS1` should not reach `host1`. Let's run Test Filters with an HTTP packet to check. This time we will specify both ACLs in the regex parameters so that we can run one check to see both results.

As with the DNS packet, `OUTSIDE_TO_INSIDE` permits the HTTP packet when it matches line `permit ip any any`. However, the input filter of `host1` has no matching line and therefore drops the packet.

In [5]:
nodes = "as2border1|host1"
acls = "OUTSIDE_TO_INSIDE|filter::INPUT"
bfq.testfilters(headers=HeaderConstraints(dstIps=host1_ip, srcIps=as1core1_ip, applications=["http"]), nodes=nodes, filters=acls).answer().frame()

,Node,Filter_Name,Flow,Action,Line_Content,Trace
0,host1,filter::INPUT,start=host1 [1.0.1.2:49152->2.128.0.101:80 TCP],DENY,default,"Flow denied by ACL named 'filter::INPUT', index 2: default"
1,as2border1,OUTSIDE_TO_INSIDE,start=as2border1 [1.0.1.2:49152->2.128.0.101:80 TCP],PERMIT,permit ip any any,"Flow permitted by 'extended ipv4 access-list' named 'OUTSIDE_TO_INSIDE', index 2: permit ip any any"


### Examining ACL behavior for other traffic from `AS1` to `host1`

We can run several more Test Filters checks programmatically to look for other permitted flows. The next cell examines what happens to some other packets from `as1core1` destined for `host1`.

The result shows that DNS packets are permitted (UDP to port 53), as well as SSH packets (TCP to port 22). We do want `AS1` devices to be able to reach `host1` via SSH, so that is expected. All other packets tested are denied by one of the two ACLs along the path to `host1`.

In [6]:
common_ports = {
    'tcp': [1, 20, 22, 25, 80, 156, 179, 389, 443, 444],
    'udp': [7, 13, 37, 42, 49, 53, 107, 123, 156, 161]
}
denied = {'tcp': [], 'udp': []}

for protocol in common_ports:
    for port in common_ports[protocol]:

        # Run Test Filters with the packet specified by protocol and port
        test_filters_result = bfq.testfilters(headers=HeaderConstraints(dstIps=host1_ip, srcIps=as1core1_ip, ipProtocols=[protocol], dstPorts=[port]), nodes=nodes, filters=acls).answer().frame()

        # Create a filtered version of the result that only contains the ACLs that denied the packet
        denies_only = test_filters_result[test_filters_result['Action'].apply(lambda action:'DENY' == action)]

        # If the filtered result is empty, then no ACLs denied the packet. Report that host1 accepted it.
        if len(denies_only) == 0:
            print("Permitted: {} on port {}".format(protocol, port))
        else:
            denied[protocol].append(port)

for protocol in denied:
    print("Denied: {} on ports {}".format(protocol, ', '.join([str(port) for port in denied[protocol]])))

Permitted: tcp on port 22
Permitted: udp on port 53
Denied: tcp on ports 1, 20, 25, 80, 156, 179, 389, 443, 444
Denied: udp on ports 7, 13, 37, 42, 49, 107, 123, 156, 161


## Ensuring all filter lines are reachable

When debugging or editing filters, it can be useful to confirm that every line is reachable -- that is, it matches some set of packets that don't match earlier lines. Often unreachable filter lines are symptomatic of past edits to the filter that did not achieve their intent.

The Filter Line Reachability question (`bfq.filterLineReachability`) identifies unreachable filter lines. Given no parameters, it will check every filter in the network, but the scope can be narrowed down using parameters `filters` and `nodes` (see [documentation](https://pybatfish.readthedocs.io/en/latest/questions.html#pybatfish.question.bfq.filterLineReachability)).

For now, let's take a look at all the filters in the network.

In [7]:
filter_line_reach_answer = bfq.filterLineReachability().answer().frame()
filter_line_reach_answer

,Sources,Unreachable_Line,Unreachable_Line_Action,Blocking_Lines,Different_Action,Reason,Additional_Info
0,[as2dept1: RESTRICT_HOST_TRAFFIC_IN],permit icmp any any,PERMIT,"[permit ip 2.128.0.0 0.0.255.255 any, deny ip any any]",True,BLOCKING_LINES,None
1,[as2dept1: RESTRICT_HOST_TRAFFIC_OUT],deny ip 1.128.0.0 0.0.255.255 2.128.0.0 0.0.255.255,DENY,[permit ip any 2.128.0.0 0.0.255.255],True,BLOCKING_LINES,None


### Examining filter line reachability results
The answer identifies two unreachable lines. Let's take a closer look at the first one, in filter `RESTRICT_HOST_TRAFFIC_IN` on node `as2dept1`. The `Unreachable_Line` column shows the unreachable line, which permits ICMP traffic. The `Blocking_Lines` column shows the two previous lines that block it, and the `Different_Action` column highlights that at least one of the blocking lines has a different action than the blocked line (in this case, deny rather than permit). The `Reason` column contains the reason why the unreachable line can't be reached. Other possible reasons for a line to be unreachable are:
- it is independently unmatchable; no packet could ever match it independent of previous lines
- it has an uncertain impact because it contains an undefined or circular reference

## Get involved with the Batfish community

Thanks for checking out our ACL debugging examples! To get involved and learn more, check out the community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTUxOTJlY2YyNTVlNGQ3MTJkOTIwZTU2YjY3YzRjZWFiYzE4ODE5ODZiNjA4NGI5NTJhZmU2ZTllOTMwZDhjMzA) and [Github](https://github.com/batfish/batfish). We would love to talk with you about Batfish or your network!